In [7]:
import matchup_calcs as mc
import tieraggregator as ta
from pokedex import Pokedex

dex = Pokedex()

I was a man on a mission. I wanted to prove that Moltres was underrated in Scarlet and Violet Overused. But how could I, a dude with a mid elo hovering in the 1400-1500 range ever properly make that argument to a higher ranked player?

A bunch of math. Because I'm a nerd. By professional training, I'm a data scientist. And I had to make sure that my method wouldn't just work for Moltres, it would have to work for *every* pokemon. And so I have learned how to definitivley quantify a pokemon's offensive and defensive weaknesses in relation to other pokemon.

Here's the basic model:

The Pokemon damage formula is pretty long, and it has the following parameters. I'm skipping the niche ones that only apply to a handful of pokemon, and the ones like level that are pretty much constant except for in a few niche cases.

1) Move power
2) Stats, using IVs and EVs
3) Weather
4) Crits
5) Random rolls
6) Burns
7) STAB
8) Type matchup

Out of these we can narrow in on a move power and stats as the only things that are really consistent from game to game because they're locked into the team builder. The dataset I got from the pokemon showdown API at https://play.pokemonshowdown.com/data/pokedex.json didn't contain movesets, so we're not counting those either.

What we're left with are the two things that basically every team is built around: a pokemon's typing and stats. But different pokemon have different matchups based on those types and stats! Blissey is very good at walling most special attackers, but is very weak into a physical fighting type. So we need our model to focus on individual offensive and defensive matchups. Step one of the model is figuring out how to generate metrics for two pokemon in a 1v1 scenario.

Here's the basic formula:

 *1v1 score = type_matchup * attacker stat / defender stat / (defender hp/86.2)*

This formula gets you a score to measure the expected damage multiplier of the attacker vs the defender based on stats and typing, and then adjusts the survivability of the defender by their hp stat. A lower defensive score is better because it means in this matchup, the pokemon is expected to take less damage. A higher offensive score is better because it means that in this matchup, the pokemon is expected to deal more damage.

In [10]:
#Here are a few example matchups from the OU tier to help you understand how the scores work and what normal scores look like.

print('Great Tusk fighting Gholdengo')
print(mc.score_1v1_names('greattusk', 'gholdengo'))
print('Raging Bolt fighting Clodsire')
print(mc.score_1v1_names('ragingbolt', 'clodsire'))
print('Skarmory fighting Clefable')
print(mc.score_1v1_names('skarmory', 'clefable'))

AttributeError: module 'matchup_calcs' has no attribute 'score_1v1_names'

Type matchup is the expected damage multiplier dealt based on the type matchup between the two pokemon. To make things easier, we're assuming that a pokemon is using a move of its type since I can't account for coverage moves easily. If the attacker is monotype this is pretty easy, but if there are dual types involved we average the expected outcomes. For example, if a Great Tusk [Ground, Fighting] is attacking a Gholdengo [Ghost, Steel], then the expected type score will be:

Ground -> [Ghost, Steel] = 2x
Fighting -> [Ghost, Steel] = 0x

[2 + 0] / 2 = 1, so we expect a type score of one.

Then the attacker stat is the attacker's highest attacking stat, and the defender stat is the defensive stat that they're using to take the hit. So in the above example, the attacker stat would be attack because Great Tusk is a physical attacker, and the defender stat is Gholdengo's physical defense. In the actual damage formula the base damage of an attack is (Base Power)*(attacker stat/defender stat).

The last part of the 1v1 score is to adjust for the defender's HP stat. We divide the overall score by (HP/86.2) because 86.2 is the average HP stat for all competitivley viable pokemon in gen 9 (meaning any pokemon in a tier PU or above, but not counting Ubers). 
